---
# this is YAML front matter
toc: true
comments: true
layout: post
title: Linux/Bash Security
description: For our Linux interaction, we decided to get hands-on with some usera and system auditing.
courses: { compsci: { week: 3 }}
categories: [C4.0]
type: hacks
---

In [1]:
%%bash


{
version=$(lsb_release -a | grep "Description" | cut -d ":" -f2 | tr -d "	")
} &>/dev/null
user=$(whoami)
kernel=$(uname -r)


echo "Output:"
echo -e "\e[40m
\e[93m                            :             .:                                                                                                          
                           .!           .^5G^.                                                                                                        
                           :Y         ..^?&@Y^..                                                                                                      
                           ~#.            ^!                                                                                                          
                           P@J                                                                                                                        
                         ^G@@@Y.                                                                                                                      
                      :!P@@@@@@#Y~.                                                                                                                   
            .::^~!7?YB&@@@@@@@@@@@&GY?7!~^::..                                                                                                        
                 ...:^!Y#@@@@@@@BJ~:....          .                                                                                                   
                        .?&@@@B~                  ~                                                  .:                                               
                          ^#@P                   :P.                                                 ~?                                               
                           7&.                .:!B@G!:.                                             ^B&!                                              
                           :5             ..:^!Y&@@@&Y!^:..                                   ..:^7G@@@@BJ~::.                                        
                           .7                   .Y@J.                                             .^J&@5~..                                           
                            ^                     Y                                                  7Y                                               
                            .                     ^                                                  .^                                               
                                                  .                                                                                                   
                                                                                                                                                      
                                                                                                                                                      
\e[35m                                       ▄███▄     ▄▄▄▄▀ ▄  █ ▄███▄   █▄▄▄▄ ▄███▄   ██   █                                                              
                                       █▀   ▀ ▀▀▀ █   █   █ █▀   ▀  █  ▄▀ █▀   ▀  █ █  █                                                              
                                       ██▄▄       █   ██▀▀█ ██▄▄    █▀▀▌  ██▄▄    █▄▄█ █                                                              
                                       █▄   ▄▀   █    █   █ █▄   ▄▀ █  █  █▄   ▄▀ █  █ ███▄                                                           
                                       ▀███▀    ▀        █  ▀███▀     █   ▀███▀      █     ▀                                                          
                                                        ▀            ▀              █                                                                 
                                                                                   ▀                                                                  
\e[93m                                                                                                                                                      
                                                                                                                                                      
                                                                                                      ..             .                                
               .:                                                                                     :^             Y~                               
               ~?                                                                                     !!         .:~P@&?:.                            
              ^B&!                                                                                    ?Y            .Y!                               
        ..:^7G@@@@BJ~::.                                                                              B#.            .                                
            .^J&@5~..                                                                               :P@@G:                                            
               7Y                                                                                 :J#@@@@&J:                                          
               .^                                                                         ..::^~!?5B@@@@@@@@@@B5?!~^^:...                             
                            .                                                            ..::^^!7YG&@@@@@@@@&GY7!~^::..                               
                            Y~                                                                    :?#@@@@#?:                .:                        
                        .:~P@&?:.                                                                   .P@@P.                  ~?                        
                           .Y!                                                                        BB                   ^B&!                       
                            .                                                                         ?J             ..:^7G@@@@BJ~::.                 
                                                                                                      ~!                 .^J&@5~..                    
                                                                                                      ::                    7Y                        
                                                                                                      ..                    .^                        
\e[35m                                                 OS: ${version}                                                                                     
                                                 Kernel: ${kernel}                                                                                  
                                                 User: ${user}                                                                             
                                                                                                                                                      
\e[97m                                                                                                                                                      "

Output:

                            :             .:                                                                                                          
                           .!           .^5G^.                                                                                                        
                           :Y         ..^?&@Y^..                                                                                                      
                           ~#.            ^!                                                                                                          
                           P@J                                                                                                                        
                         ^G@@@Y.                                                                                                                      
                      :!P@@@@@@#Y~.                                                  